### Import modules

In [1]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3
import json

### Initialize and configure OpenSearch client

In [2]:
with open('host.txt', 'r') as fp:
    host = fp.read()
region = "us-east-1"
service = "aoss"
session = boto3.Session(profile_name="rapid-innovation-dev")
credentials = session.get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

In [3]:
client = OpenSearch(
    hosts = [{"host": host, "port": 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    pool_maxsize = 20
)

### Initialize and configure Boto Client for Bedrock

In [4]:
bedrock = session.client(
 service_name='bedrock-runtime',
 region_name='us-east-1',
)

### Define function for generating embeddings

In [5]:
def text_embedding(text):
    body=json.dumps({"inputText": text})
    response = bedrock.invoke_model(body=body, modelId='amazon.titan-embed-text-v1', accept='application/json', contentType='application/json')
    response_body = json.loads(response.get('body').read())
    embedding = response_body.get('embedding')
    return embedding

### Create the prompt

In [6]:
prompt='Who won the Best Actor award in a supporting role?'

### Build context

In [7]:
def search_index(vector):
    document = {
        "size": 15,
        "_source": {"excludes": ["nominee_vector"]},
        "query": {
            "knn": {
                 "nominee_vector": {
                     "vector": vector,
                     "k":15
                 }
            }
        }
    }
    response = client.search(
    body = document,
    index = "oscars-index"
    )
    return response

In [8]:
vector=text_embedding(prompt)

In [9]:
response=search_index(vector)
data=response['hits']['hits']

In [10]:
data

[{'_index': 'oscars-index',
  '_id': '1%3A0%3AkOGmJZABlACFB3AV_gFM',
  '_score': 0.7535921,
  '_source': {'nominee_text': 'Paul Mescal got nominated under the category, actor in a leading role, for the film Aftersun but did not win'}},
 {'_index': 'oscars-index',
  '_id': '1%3A0%3AocOnJZABw5U0TIJoBKi5',
  '_score': 0.7324267,
  '_source': {'nominee_text': 'Brian Tyree Henry got nominated under the category, actor in a supporting role, for the film Causeway but did not win'}},
 {'_index': 'oscars-index',
  '_id': '1%3A0%3AoMOnJZABw5U0TIJoAKhR',
  '_score': 0.7311648,
  '_source': {'nominee_text': 'Bill Nighy got nominated under the category, actor in a leading role, for the film Living but did not win'}},
 {'_index': 'oscars-index',
  '_id': '1%3A0%3Ak-GnJZABlACFB3AVCwEe',
  '_score': 0.72693133,
  '_source': {'nominee_text': 'Ke Huy Quan got nominated under the category, actor in a supporting role, for the film Everything Everywhere All at Once to win the award'}},
 {'_index': 'oscars-

In [11]:
context = ''
for item in data:
    context += item['_source']['nominee_text'] + '\n'
print(context)

Paul Mescal got nominated under the category, actor in a leading role, for the film Aftersun but did not win
Brian Tyree Henry got nominated under the category, actor in a supporting role, for the film Causeway but did not win
Bill Nighy got nominated under the category, actor in a leading role, for the film Living but did not win
Ke Huy Quan got nominated under the category, actor in a supporting role, for the film Everything Everywhere All at Once to win the award
Colin Farrell got nominated under the category, actor in a leading role, for the film The Banshees of Inisherin but did not win
Brendan Fraser got nominated under the category, actor in a leading role, for the film The Whale to win the award
Brendan Gleeson got nominated under the category, actor in a supporting role, for the film The Banshees of Inisherin but did not win
Barry Keoghan got nominated under the category, actor in a supporting role, for the film The Banshees of Inisherin but did not win
Judd Hirsch got nominat

In [12]:
augmented_prompt=f'Context - {context}\nBased on the above context, answer this question - {prompt}'

In [13]:
augmented_prompt

'Context - Paul Mescal got nominated under the category, actor in a leading role, for the film Aftersun but did not win\nBrian Tyree Henry got nominated under the category, actor in a supporting role, for the film Causeway but did not win\nBill Nighy got nominated under the category, actor in a leading role, for the film Living but did not win\nKe Huy Quan got nominated under the category, actor in a supporting role, for the film Everything Everywhere All at Once to win the award\nColin Farrell got nominated under the category, actor in a leading role, for the film The Banshees of Inisherin but did not win\nBrendan Fraser got nominated under the category, actor in a leading role, for the film The Whale to win the award\nBrendan Gleeson got nominated under the category, actor in a supporting role, for the film The Banshees of Inisherin but did not win\nBarry Keoghan got nominated under the category, actor in a supporting role, for the film The Banshees of Inisherin but did not win\nJudd

### Invoke Titan LLM Endpoint

In [14]:
config={
      "maxTokenCount": 1000,
      "stopSequences": [],
      "temperature":0.1,
      "topP":1
}

body = json.dumps({'inputText': augmented_prompt,'textGenerationConfig':config})

In [15]:
response = bedrock.invoke_model( 
 modelId='amazon.titan-tg1-large', 
 body=body
)

In [16]:
response_body = json.loads(response.get('body').read())
print(response_body.get('results')[0].get('outputText'))


Ke Huy Quan won the Best Actor award in a supporting role.
